# OBJECTNESS EVALUATION

The purpose of this Notebook is to evaluate the performance of different generic object proposal approaches on high-resultion satellite data

In [ ]:
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader

import datasets
from helpers import io

## Settings

In [ ]:
DATA_PATH = "/home/jovyan/work/mydata/DENMARK/512x512"

## Load Data

In [ ]:
images_path = os.path.join(DATA_PATH, 'image_sets_trees', 'eval_shapes.txt')
#images_path = os.path.join(DATA_PATH, 'image_sets_buildings', 'eval_shapes.txt')
images = [name.replace("\n","") for name in io.readText(images_path)]

In [ ]:
dataset = datasets.getDataset(name = "denmark_shapes", 
                              path = DATA_PATH,
                              images = images,
                              object_type = 'trees_eval', #trees_eval or buildings
                              n_classes = 2,
                              transform = None)

sampler = torch.utils.data.RandomSampler(dataset)
loader = DataLoader(dataset, sampler = sampler, batch_size = 1, drop_last = True, num_workers = 1)

print(f"Loaded eval set of {len(loader)} images...")

## Eval COB

In [ ]:
from models.cobnet import COBNet
from helpers.cob.dataset import COBtransform

means = [0.492, 0.475, 0.430]
stds = [0.176, 0.173, 0.176]
transform = COBtransform(means, stds, 512)

cob_model = COBNet()
cob_model.load_state_dict(torch.load("/home/jovyan/work/runs/X_COBNET/cp_or.pth.tar"))

In [ ]:
cob_loss_running = 0
cob_mIoU_running = 0
cob_dice_running = 0
cob_pAcc_running = 0

for batch in tqdm(loader):
    image = batch["images"].numpy()
    shapes = batch["shapes"].squeeze().numpy()
    
    # transforms
    image_cob = transform(images = image)[0]
    image_cob = np.stack((image_cob[:,:,0], image_cob[:,:,1], image_cob[:,:,2]), axis = 0)
    cob_tensor = torch.tensor(image_cob[np.newaxis, ...]).float()
    
    # model run
    cob_model.eval()
    with torch.no_grad():
        out_cob = cob_model(cob_tensor)
    
    cob_preds = out_cob['y_fine'].sigmoid().squeeze()
    
    # eval with BCELoss
    criterion = torch.nn.BCELoss()
    loss1 = criterion(cob_preds, torch.from_numpy(shapes.astype(np.float32)))
    loss0 = criterion(1 - cob_preds, torch.from_numpy(shapes.astype(np.float32)))
    ### NOTE: shape area should be as homogeneous as possible, not as low/high as possible -> take better score
    if loss1 < loss0:
        cob_loss_running += loss1.item()
    else:
        cob_loss_running += loss0.item()
    
    # eval with mIoU
    shapes_mask = shapes == 1
    cob_preds_np = torch.round(cob_preds).numpy()
    cob_mask1 = cob_preds_np == 1
    cob_mask0 = cob_preds_np == 0
    intersection1 = np.logical_and(cob_mask1, shapes_mask).sum()
    intersection0 = np.logical_and(cob_mask0, shapes_mask).sum()
    
    if intersection1 > intersection0:
        union = np.logical_or(cob_mask1, shapes_mask).sum()
        cob_mIoU_running += intersection1 / union
        total = cob_mask1.sum() + shapes_mask.sum()
        cob_dice_running = (2 * intersection1) / total
    else:
        union = np.logical_or(cob_mask0, shapes_mask).sum()
        cob_mIoU_running += intersection0 / union
        total = cob_mask0.sum() + shapes_mask.sum()
        cob_dice_running = (2 * intersection1) / total
    
    correct1 = (cob_mask1 == shapes_mask).sum()
    correct0 = (cob_mask0 == shapes_mask).sum()
    
    if correct1 > correct0:
        pixels = np.size(cob_mask1)
        cob_pAcc_running = correct1 / pixels
    else:
        pixels = np.size(cob_mask0)
        cob_pAcc_running = correct0 / pixels
        

cob_loss = cob_loss_running / len(loader)
cob_mIoU = cob_mIoU_running / len(loader)
cob_dice = cob_dice_running / len(loader)
cob_pAcc = cob_pAcc_running / len(loader)
print(f'COB Scores: \n -BCELoss: {cob_loss}\n -mIoU: {cob_mIoU}\n -Dice: {cob_dice}\n -pAcc: {cob_pAcc}')

## Eval WTP

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath('/home/jovyan/work/ma/helpers/objectness', nargout=0)
eng.addpath('/home/jovyan/work/ma/helpers/objectness/pff_segment', nargout=0)
eng.addpath('/home/jovyan/work/ma/helpers/objectness/MEX', nargout=0)

In [ ]:
wtp_loss_running = 0
wtp_mIoU_running = 0
wtp_dice_running = 0
wtp_pAcc_running = 0

for batch in tqdm(loader):
    image = batch['meta']['path'][0]
    shapes = batch["shapes"].squeeze().numpy()
    
    # matlab function run
    wtp_preds = np.asarray(eng.getHeatMap(image, 100))
    wtp_preds = torch.from_numpy(wtp_preds.astype(np.float32))
    
    # eval with BCELoss
    criterion = torch.nn.BCELoss()
    loss1 = criterion(wtp_preds, torch.from_numpy(shapes.astype(np.float32)))
    loss0 = criterion(1 - wtp_preds, torch.from_numpy(shapes.astype(np.float32)))
    ### NOTE: shape area should be as homogeneous as possible, not as low/high as possible -> take better score
    if loss1 < loss0:
        wtp_loss_running += loss1.item()
    else:
        wtp_loss_running += loss0.item()
        
    # eval with mIoU and Dice
    shapes_mask = shapes == 1
    wtp_preds_np = torch.round(wtp_preds).numpy()
    wtp_mask1 = wtp_preds_np == 1
    wtp_mask0 = wtp_preds_np == 0
    intersection1 = np.logical_and(wtp_mask1, shapes_mask).sum()
    intersection0 = np.logical_and(wtp_mask0, shapes_mask).sum()
    
    if intersection1 > intersection0:
        union = np.logical_or(wtp_mask1, shapes_mask).sum()
        wtp_mIoU_running += intersection1 / union
        total = wtp_mask1.sum() + shapes_mask.sum()
        wtp_dice_running = (2 * intersection1) / total
        
    else:
        union = np.logical_or(wtp_mask0, shapes_mask).sum()
        wtp_mIoU_running += intersection0 / union
        total = wtp_mask0.sum() + shapes_mask.sum()
        wtp_dice_running = (2 * intersection0) / total
        
    correct1 = (wtp_mask1 == shapes_mask).sum()
    correct0 = (wtp_mask0 == shapes_mask).sum()
    
    if correct1 > correct0:
        pixels = np.size(wtp_mask1)
        wtp_pAcc_running = correct1 / pixels
    else:
        pixels = np.size(wtp_mask0)
        wtp_pAcc_running = correct0 / pixels
        
    
wtp_loss = wtp_loss_running / len(loader)
wtp_mIoU = wtp_mIoU_running / len(loader)
wtp_dice = wtp_dice_running / len(loader)
wtp_pAcc = wtp_pAcc_running / len(loader)
print(f'WTP Scores: \n -BCELoss: {wtp_loss}\n -mIoU: {wtp_mIoU}\n -Dice: {wtp_dice}\n -pAcc: {wtp_pAcc}')

In [ ]:
eng.quit()

## Eval CAM

In [ ]:
from torchvision.models import resnet50
from torchcam.cams import SmoothGradCAMpp

cam_model = resnet50(pretrained = True).eval()
cam_extractor = SmoothGradCAMpp(cam_model)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = [0.492, 0.475, 0.430], std = [0.176, 0.173, 0.176])])
resize = transforms.Resize((512, 512))

In [ ]:
cam_loss_running = 0
cam_mIoU_running = 0
cam_dice_running = 0
cam_pAcc_running = 0

for batch in tqdm(loader):
    image = batch['images'].squeeze().numpy()
    shapes = batch["shapes"].squeeze().numpy()
    image = transform(image)
    
    # run estimation method
    out = cam_model(image.unsequeeze(0)) # needs to return [height x width] matrix in 0 - 1 range (sigmoid) as tensor
    cam_preds = cam_extractor(out.squeeze(0).argmax().item(), out)
    res = resize(cam_preds.unsqueeze(0)).squeeze()
    
    # eval with BCELoss
    criterion = torch.nn.BCELoss()
    loss1 = criterion(cam_preds, torch.from_numpy(shapes.astype(np.float32)))
    loss0 = criterion(1 - cam_preds, torch.from_numpy(shapes.astype(np.float32)))
    ### NOTE: shape area should be as homogeneous as possible, not as low/high as possible -> take better score
    if loss1 < loss0:
        cam_loss_running += loss1.item()
    else:
        cam_loss_running += loss0.item()
        
    # eval with mIoU
    shapes_mask = shapes == 1
    cam_preds_np = torch.round(cam_preds).numpy()
    cam_mask1 = cam_preds_np == 1
    cam_mask0 = cam_preds_np == 0
    intersection1 = np.logical_and(cam_mask1, shapes_mask).sum()
    intersection0 = np.logical_and(cam_mask0, shapes_mask).sum()
    
    if intersection1 > intersection0:
        union = np.logical_or(cam_mask1, shapes_mask).sum()
        cam_mIoU_running += intersection1 / union
        total = cam_mask1.sum() + shapes_mask.sum()
        cam_dice_running = (2 * intersection1) / total
    else:
        union = np.logical_or(cam_mask0, shapes_mask).sum()
        cam_mIoU_running += intersection0 / union
        total = cam_mask0.sum() + shapes_mask.sum()
        cam_dice_running = (2 * intersection0) / total
        
    correct1 = (cam_mask1 == shapes_mask).sum()
    correct0 = (cam_mask0 == shapes_mask).sum()
    
    if correct1 > correct0:
        pixels = np.size(cam_mask1)
        cam_pAcc_running = correct1 / pixels
    else:
        pixels = np.size(cam_mask0)
        cam_pAcc_running = correct0 / pixels
        
    
cam_loss = cam_loss_running / len(loader)
cam_mIoU = cam_mIoU_running / len(loader)
cam_dice = cam_dice_running / len(loader)
cam_pAcc = cam_pAcc_running / len(loader)
print(f'CAM Scores: \n -BCELoss: {cam_loss}\n -mIoU: {cam_mIoU}\n -Dice: {cam_dice}\n -pAcc: {cam_pAcc}')

----------------------------------------------------------------------------------------------------------------------------------------------------------------
# EXPERIMENTATION ZONE

In [4]:
dataiter = iter(train_loader)
batch = dataiter.next()

In [6]:
image = batch["images"].squeeze().numpy()
#shapes = batch["shapes"].squeeze().numpy()
labels = batch["label"].long().cuda()

In [ ]:
transform_mean = [0.492, 0.475, 0.430] # from preprocessing
transform_std = [0.176, 0.173, 0.176]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = transform_mean, std = transform_std)])
resize = transforms.Resize((512, 512))

In [ ]:
image = transform(image)
out = cam_model(image.unsqueeze(0))
cam_preds = cam_extractor(out.squeeze(0).argmax().item(), out)
res = resize(cam_preds.unsqueeze(0)).squeeze()

In [ ]:
# to visualize CAM prediction
from matplotlib import cm
Image.fromarray(np.uint8(cm.jet(res) * 255))

In [ ]:
# to visualize objectness measure heatmap
for image in images:
    name = image+".jpg"
    path = os.path.join("/home/jovyan/work/mydata/DENMARK/512x512/images", name)
    wtp_preds = np.asarray(eng.getHeatMap(path, 100))
    img = Image.fromarray(np.uint8(wtp_preds * 255), 'L')
    img.save(name)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------
# OPTIONAL: TRAIN OWN MODEL FOR CAM PREDICTIONS

In [1]:
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50

import datasets
from helpers import io

In [2]:
DATA_PATH = "/home/jovyan/work/processed/256x256"
object_type = "buildings"

images_path = os.path.join(DATA_PATH, 'image_sets_'+object_type, 'all.txt')
images_list = [name.replace("\n","") for name in io.readText(images_path)]

train_size = round(len(images_list) * 0.8)
train_images = images_list[:train_size]
val_size = round(len(images_list) * 0.1)
val_images = images_list[train_size:(train_size + val_size)]

# create transformation object
transform_mean = [0.492, 0.475, 0.430] # from preprocessing
transform_std = [0.176, 0.173, 0.176]

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean = transform_mean, 
                                                     std = transform_std)])

train_set = datasets.getDataset(name = "denmark_points", 
                              path = DATA_PATH,
                              images = train_images,
                              object_type = object_type,
                              n_classes = 2,
                              transform = transform)

train_sampler = torch.utils.data.RandomSampler(train_set)
train_loader = DataLoader(train_set, sampler = train_sampler, batch_size = 64, drop_last = True, num_workers = 1)

val_set = datasets.getDataset(name = "denmark_points", 
                              path = DATA_PATH,
                              images = val_images,
                              object_type = object_type,
                              n_classes = 2,
                              transform = transform)

val_sampler = torch.utils.data.RandomSampler(val_set)
val_loader = DataLoader(val_set, sampler = val_sampler, batch_size = 64, drop_last = True, num_workers = 1)

print(f"Loaded training set of {len(train_loader)} batches and validation set of {len(val_loader)} batches...")

Loaded training set of 1700 batches and validation set of 212 batches...


In [3]:
model = resnet50(pretrained = True)

for param in model.parameters():
    param.requires_grad = False
    
model.fc = torch.nn.Sequential(torch.nn.Linear(2048, 512),
                         torch.nn.ReLU(),
                         torch.nn.Dropout(0.2),
                         torch.nn.Linear(512, 2),
                         torch.nn.LogSoftmax(dim=1))
model.to("cuda")

criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.003)

In [ ]:
epochs = 25
train_losses, val_losses = [], []

for epoch in tqdm(range(epochs)):
    running_loss = 0
    val_loss = 0
    accuracy = 0
    
    model.train()
    for batch in tqdm(train_loader):
        
        inputs = batch["images"].cuda()
        labels = batch["label"].long().cuda()
        
        optimizer.zero_grad()
        out = model.forward(inputs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    model.eval()
    with torch.no_grad():
        for batch in tqdm(val_loader):
            
            inputs = batch["images"].cuda()
            labels = batch["label"].long().cuda()
            
            out = model.forward(inputs)
            batch_loss = criterion(out, labels)
            
            val_loss += batch_loss.item()

            ps = torch.exp(out)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
    train_losses.append(running_loss/len(train_loader))
    val_losses.append(val_loss/len(val_loader)) 
    
    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {running_loss/len(train_loader):.3f}.. "
          f"Test loss: {val_loss/len(val_loader):.3f}.. "
          f"Test accuracy: {accuracy/len(val_loader):.3f}")

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1700 [00:00<?, ?it/s]